Ring Polymer Melt
===========

insert description

### Concepts Used
- blergh

### Tools Used
- blergh

References
-----------
 
  

Global Imports and Setup
----------------------------

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh')

%load_ext autoreload
%autoreload 2

In [ ]:
%opts Curve Scatter [width=500,height=400] Layout [shared_axes=False] Scatter (size=10,alpha=0.5)
%opts Curve Scatter [fontsize={'xlabel':14,'xlabel':14,'ylabel':14,'ticks':12}]
%opts Overlay [legend_position='bottom_left']
%opts Layout [shared_axes=False]


colors = {}
colors[2000] = 'blue'
colors[500] = 'red'
colors[100] = 'green'

ls = {}
ls[2000] = 'solid'
ls[500] = 'dashed'
ls[100] = 'dotted'

markers = {}
markers[2000] = 'o'
markers[500] = '^'
markers[100] = 'd'

Demo
-----

Now we do the PRISM calculation...

In [ ]:
import typyPRISM
from typyPRISM.calculate.prism.pair_correlation import pair_correlation


dA = 1.0
dB = 1.0
vA = 4.0/3.0 * np.pi * (dA/2.0)**(3.0)
vB = 4.0/3.0 * np.pi * (dB/2.0)**(3.0)

eta = 0.3
phi = 0.5
rhoA = phi * eta/vA
rhoB = (1-phi) * eta/vA

lengthA = 2000
lengthB = 2000

sys = typyPRISM.System(['A','B'],kT=1.0)
sys.domain = typyPRISM.Domain(dr=0.05,length=2048)

sys.density['A'] = rhoA
sys.density['B'] = rhoB

sys.closure['A','A'] = typyPRISM.closure.PercusYevick()
sys.closure['A','B'] = typyPRISM.closure.PercusYevick()
sys.closure['B','B'] = typyPRISM.closure.PercusYevick()

sys.potential['A','A'] = typyPRISM.potential.HardSphere(sigma=dA)
sys.potential['A','B'] = typyPRISM.potential.HardCoreLennardJones(sigma=(dA+dB)/2.0,epsilon=-0.1)
# sys.potential['A','B'] = typyPRISM.potential.HardSphere(sigma=(dA+dB)/2.0)
sys.potential['B','B'] = typyPRISM.potential.HardSphere(sigma=dB)

sys.omega['A','A'] = typyPRISM.omega.GaussianRing(sigma=1.0,length=lengthA)
sys.omega['A','B'] = typyPRISM.omega.InterMolecular()
sys.omega['B','B'] = typyPRISM.omega.GaussianRing(sigma=1.0,length=lengthB)

PRISM = sys.createPRISM()

result = PRISM.solve()


Here we reproduce Figure 1 of Reference 1.

In [ ]:
%%opts Overlay [legend_position='bottom_right']
from math import sqrt

extents = (0,0,13,1.0)

gr_plots = []
for name,N,x,y in gr_results:
    label = 'N={} (typyPRISM)'.format(N)
    style = {'line_dash':ls[N],'color':colors[N]}
    c1 = hv.Curve((x,y),label=label,extents=extents)(style=style)
    gr_plots.append(c1)
    
for N,x,y in gr_compare:
    label = 'N={} (Ref 1)'.format(N)
    style = {'marker':markers[N],'color':colors[N]}
    c1 = hv.Scatter((x,y),label=label,extents=extents)(style=style)
    gr_plots.append(c1)
    
    
hv.Overlay(gr_plots).redim.label(x='r',y='g(r)')